In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:
action=[(0,0), (0,1), (1,1)]

Number_level = 10;
#Transition_matrix = [0.86 0.14 0.0 0.0; 0.0 0.83 0.17 0.0;0.0 0.0 0.89 0.11;1.0 0.0 0.0 0.0];
fullname = "tm10";
# generate_transition_matrix(Number_level, fullname) 
Transition_matrix = load(fullname*".jld2","transition_matrix");
failure_penalty = -1200.0;
maintenance_penalty = -100.0;
setup_cost = -300.0;
normal_operation = 0.0;
system_penalty=-2000

-2000

In [3]:
function reward(s, a,s1=10, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
    if a==1 #NO REPAIR, No SYSTEM FAILURE
         if s==Number_level
             r=system_penalty/(limit);
             return r
         else
             crd = Categorical(T[s,:,1]);
             s = rand(crd);
             r = normal_operation ;
              return r
             end
         end
     if a==2 # UNIT NOT REPAIRED UNLESS FAILURE BUT REPAIR IN SYSTEM
         if s==Number_level
              s=1
             crd = Categorical(T[s,:,1]);
             s = rand(crd);
             r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
         return r
         else
             crd = Categorical(T[s,:,1]);
             s = rand(crd);
             r = normal_operation +  setup_cost/NumberUnits;
              return r
         end
     end
 
      if a==3 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM NO SYSTEM FAILURE
         if s==Number_level
              s=1
             crd = Categorical(T[s,:,1]);
             s = rand(crd);
             r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
         return r
         else
         s1=deepcopy(s)
         s=1
         r = maintenance_penalty +  setup_cost/NumberUnits
          crd = Categorical(T[s,:,1]);
             s = rand(crd);
             r+= normal_operation  ;
 #            r*=(s1/10);
        return r
 #         end
     end
     end
     
     if a==4 #UNIT REPAIRED AND MAINTENANCE IN SYSTEM with SYSTEM FAILURE FAILED UNIT TAKES Kth Component
         if s==Number_level
              s=1
             crd = Categorical(T[s,:,1]);
             s = rand(crd);
             r=failure_penalty +  setup_cost/NumberUnits + normal_operation + system_penalty/(limit) ;
         return r
         else
         s1=deepcopy(s)
         s=1
         r = maintenance_penalty +  setup_cost/NumberUnits
          crd = Categorical(T[s,:,1]);
             s = rand(crd);
             r+= normal_operation  ;
 #            r*=(s1/10);
        return r
 #         end
     end
     end
     
     end

reward (generic function with 3 methods)

In [4]:
s=1:Number_level;

In [5]:
(Transition_matrix)

10×10 Matrix{Float64}:
 0.269528  0.168607  0.130661  …  0.0483297  0.00607726  0.00466985
 0.0       0.332736  0.174857     0.0483409  0.0133439   0.00507506
 0.0       0.0       0.218499     0.0823996  0.0244349   0.00601449
 0.0       0.0       0.0          0.0926184  0.0592904   0.00740484
 0.0       0.0       0.0          0.174542   0.0815024   0.0129094
 0.0       0.0       0.0       …  0.254279   0.105602    0.0195799
 0.0       0.0       0.0          0.271986   0.158174    0.0867437
 0.0       0.0       0.0          0.589988   0.260696    0.149316
 0.0       0.0       0.0          0.0        0.519525    0.480475
 1.0       0.0       0.0          0.0        0.0         0.0

In [6]:
T=zeros(Number_level,Number_level,3);


In [7]:
((1:10)./10)

0.1:0.1:1.0

In [8]:
T[:,:,1].=Transition_matrix;

In [9]:
T[:,:,2].=Transition_matrix;


In [10]:
for j in 1:Number_level
        T[j,:,3].=Transition_matrix[1,:];

        end
# for j in 1:10
#         T[j,:,3].=0.0
#     T[j,1,3]=1.0
#         end

In [11]:
T[Number_level,:,1].=0.0
T[Number_level,Number_level,1]=1.0
T[Number_level,:,2].=Transition_matrix[1,:]


10-element view(::Array{Float64, 3}, 10, :, 2) with eltype Float64:
 0.2695279572108137
 0.16860741623354286
 0.1306613209439592
 0.09857168678543618
 0.09171796107138079
 0.09134806335671229
 0.09048875946531719
 0.04832973229444604
 0.00607725632113932
 0.004669846317252394

In [12]:
# T[:,:,1].=T[:,:,1].*(repeat(.5:.5,10)-((10:1)./20))
# T[:,:,2].=T[:,:,2].*(repeat(.5:.5,10)-((10:1)./20))

In [13]:
T

10×10×3 Array{Float64, 3}:
[:, :, 1] =
 0.269528  0.168607  0.130661  …  0.0483297  0.00607726  0.00466985
 0.0       0.332736  0.174857     0.0483409  0.0133439   0.00507506
 0.0       0.0       0.218499     0.0823996  0.0244349   0.00601449
 0.0       0.0       0.0          0.0926184  0.0592904   0.00740484
 0.0       0.0       0.0          0.174542   0.0815024   0.0129094
 0.0       0.0       0.0       …  0.254279   0.105602    0.0195799
 0.0       0.0       0.0          0.271986   0.158174    0.0867437
 0.0       0.0       0.0          0.589988   0.260696    0.149316
 0.0       0.0       0.0          0.0        0.519525    0.480475
 0.0       0.0       0.0          0.0        0.0         1.0

[:, :, 2] =
 0.269528  0.168607  0.130661  …  0.0483297  0.00607726  0.00466985
 0.0       0.332736  0.174857     0.0483409  0.0133439   0.00507506
 0.0       0.0       0.218499     0.0823996  0.0244349   0.00601449
 0.0       0.0       0.0          0.0926184  0.0592904   0.00740484
 0.0      

In [14]:
T[:, :, 3]

10×10 Matrix{Float64}:
 0.269528  0.168607  0.130661  …  0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661  …  0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985

In [15]:
function explore()
    c=[1/3,1/3,1/3]

     crd = Categorical(c);
     return rand(crd)
    end 
function generative(s, a, rng=23)       # calculate true component-level reward r_i(s_i,a_i) nd get evolving states
    if a==1 # No replacement in system, no system failure
        # if s==Number_level
        #     r=system_penalty/(limit);
        #     return (sp=s, r=r)
        # else
            crd = Categorical(T[s,:,1]);
            s = rand(crd);
            r = normal_operation ;
             return (sp=s, r=r)
            # end
        end
    if a==2 # replacement in system, no system failure, corrective replacement for unit
        if s==Number_level
             s=1
            crd = Categorical(T[s,:,2]);
            s = rand(crd);
            #r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
            r=failure_penalty  + normal_operation  ;
        return (sp=s, r=r)
        else
            crd = Categorical(T[s,:,2]);
            s = rand(crd);
            #r = normal_operation +  setup_cost/NumberUnits;
            r = normal_operation;
             return (sp=s, r=r)
        end
    end

     if a==3 # replacement in system, no system failure, preventive replacement for unit
        if s==Number_level
             s=1
            crd = Categorical(T[s,:,3]);
            s = rand(crd);
            #r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
            r=failure_penalty + normal_operation  ;
        return (sp=s, r=r)
        else
        s1=deepcopy(s)
        s=1
        #r = maintenance_penalty +  setup_cost/NumberUnits
        r = maintenance_penalty
         crd = Categorical(T[s,:,3]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
       return (sp=s, r=r)
#         end
    end
    end
    
    
    if a==4 # replacement in system, system failure occurs, preventive replacement for unit
        if s==Number_level
             s=1
            crd = Categorical(T[s,:,3]);
            s = rand(crd);
            #r=failure_penalty +  setup_cost/NumberUnits + normal_operation + system_penalty/(limit) ;
            r=failure_penalty+ normal_operation;
        return (sp=s, r=r)
        else
        s1=deepcopy(s)
        s=1
        #r = maintenance_penalty +  setup_cost/NumberUnits
        r = maintenance_penalty
         crd = Categorical(T[s,:,3]);
            s = rand(crd);
            r+= normal_operation;
#            r*=(s1/10);
       return (sp=s, r=r)
#         end
    end
    end

    end

generative (generic function with 2 methods)

In [16]:
function ValueIteration(S, T, γ, p1=0.01, d=100)
    Q=zeros((Number_level,4))
#     print(Q)
    V= zeros(Float64, (Number_level)).+1000
    diff=0
    i=0

    
   
    while i<=10000
#     while  i<=10000
    for s in 1:Number_level
       Threads.@threads for a in 1:4
#                 println(a)
#                 println(reward(s,a))
                if a>3
                Q[s,a]= (reward(s,a)) + γ*( sum(T[s,s1,3]*(V[s1]) for (s1) in 1:Number_level ))
                else
                Q[s,a]= (reward(s,a)) + γ*( sum(T[s,s1,a]*(V[s1]) for (s1) in 1:Number_level ))
                end
        end
#             print("here")

                p1=[exp(lamb*sum(Q[s,a3])) for a3 in 1:4]
                p1=p1./sum(p1)
#                  V[s]=sum(p1[1:3])*(p[1]*Q[s,1] + (p[2]+p[3])*max(Q[s,2],Q[s,3])) + sum(p1[4:(NumberUnits-limit+4)])*(sum(p2[b]*Q[s,b+3] for b in 1:(NumberUnits-limit+1)))
                 V[s]=(p1[1]*Q[s,1] + p1[2]*Q[s,2] +p1[3]*Q[s,3]+p1[4]*Q[s,4])
#             println(findmax(Q[s,1:4]))
#                 println(Q)
#                 V[s]=findmax(Q[s,1:4])[1]
                 
        end

        i+=1
        end
    return Q
    end

ValueIteration (generic function with 3 methods)

In [17]:

# function decison(Q, s)
#    a_final=[]
#    if Number_level in s
# #         print("hi")
#         d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#         d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
#         min=-1e+10
#         opt=0
#         for (i,j) in enumerate(d)
#             if min < sum((j))
#                 min=sum((j))
#                 opt=i
#             end
#         end
#         return d1[opt]
#     end
#     q0=0.0
#     a_0=repeat(1:1,NumberUnits);
#     for i in 1:NumberUnits
#             q0+=Q[s[i],1]
#     end
#     d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#     d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
# #     d[1]=a_0
# #     print("here")
# #     d1[1]=(1,1,1,1,1,1,1,1,1,1)
#     min=-1e+10
#     opt=0
#     for (i,j) in enumerate(d)
#         if i==1
#             if min < q0
#             min=q0
#             opt=i
#         end
#         elseif min < sum((j))
#             min=sum((j))
#             opt=i
#         end
#     end
#     if opt==1
#         return a_0
#     else
#         return d1[opt]
#     end
#     return opt
# end


function decison(Q, s,n=0.2)
   a_final=[]
    alpha=count(i->(i==Number_level), s) 
    alpha=deepcopy(limit)
   if count(i->(i==Number_level), s) >=limit
#         print("hi")
        count_1=0
        a_1=repeat(1:1,NumberUnits);
        for i in 1:NumberUnits
            a_1[i]=argmax(a->Q[s[i],a], [2,3])
            if s[i]==Number_level && alpha>0
                a_1[i]=4
                alpha=alpha-1
            end
        end
        return a_1
    end
    
    
    
    
    
    q0=0.0
    a_0=repeat(1:1,NumberUnits);
    for i in 1:NumberUnits
            q0+=Q[s[i],1]
    end
    a_1=repeat(1:1,NumberUnits);
    b=0
    for i in 1:NumberUnits
            a_1[i]=argmax(a->Q[s[i],a], [2,3])
            if s[i]==Number_level
                a_1[i]=3
            end
    end
    q1=0.0
    for i in 1:NumberUnits
            q1+=Q[s[i],a_1[i]]
    end
    
    if !(3 in a_1) 
        a_1=a_0
        q1=q0
    end
    
    
     if q0>q1
            return a_0
        else
#             print("HI")
            return a_1
            
        end
end

decison (generic function with 2 methods)

In [18]:

using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],lamb=[],mean=[],std=[])
cost=[ [0	-200	-100	-1200 -2000],
    [0	-400	-100	-1200 -2000],
    [0	-600	-100	-1200 -2000],
    [0	-800	-100	-1200 -2000],
    [0	-1000	-100	-1200 -2000],
    [0	-1200	-100	-1200 -2000],
    [0	-1400	-100	-1200 -2000],
    [0	-1600	-100	-1200 -2000],
    [0	-1800	-100	-1200 -2000],
    [0	-2000	-100	-1200 -2000],
    [0  -2400   -100    -1200 -2000],
    [0  -2800   -100    -1200 -2000],
    [0  -3200   -100    -1200 -2000]
] 


for units in [60,50,40,30,25,20]
    for K in 2:2:convert(Int64, floor((units+1)/2))
        
    global NumberUnits=convert(Int64,units)
        global limit=convert(Int64,K)
#      for lambda in [1,0.9,0.75,0.5,0.1,0.075,0.05,0.01,0.0075,0.005,0.0001]
     for lambda in [100,10,5,1,0.5, 0.1,0.05,1e-2,0.5e-2,1e-3,0.5e-3,1e-4,0.5e-4,1e-5,0.5e-5,1e-6,0.5e-6,1e-7,0.5e-7,1e-8]
#      for lambda1 in [0.0001,0.00001]
        
        global lamb=lambda
#         global lamb1=lambda1
            
            for c in cost
        tick()
   println(units)  
             print("K : ")
                    println(K)
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
     println(c)   

    global NumberUnits=units
     Q=ValueIteration(s,T,.95,3000)
#                 println(Q)
            println(isnan(Q[1,1]))
       if sum(isnan(Q[1,1]))>0.0
            continue
        end
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads  for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
            r2=0.0
#     print(s)
#                         println(a)
global failed_components = 0
for i in 1:NumberUnits
#             print(generative(s[i],a[i],234))
    if s[i] == Number_level
        failed_components += 1
    end
    s[i],r[i]=generative(s[i],a[i],234);
    r2+=r[i]
end 
if  (2 in a) ||(3 in a) ||(4 in a) 
    # if any replacement
        r2+=setup_cost
                end
if failed_components >= limit
    r2+=system_penalty
end
    s=s;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
#     println(rk)
    results[h]=rk
#     println(results)
end
println(units)
            print("k : ")
            println(limit)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end
end
end

60
K : 2

[ Info:  started timer at: 2024-03-14T07:31:44.651



[0 -200 -100 -1200 -2000]
true
60
K : 2
[0 -400 -100 -1200 -2000]


[ Info:  started timer at: 2024-03-14T07:32:02.348


true
60
K : 2
[0 -600 -100 -1200 -2000]


...

60
k : 14
0.0005
-72504.38002260435
2714.4372076742443


[ Info:           27.3150097s: 27 seconds, 315 milliseconds
[ Info:  started timer at: 2024-03-15T00:53:26.348


60
K : 14
[0 -1800 -100 -1200 -2000]
false


[ Info:  started timer at: 2024-03-15T00:53:43.501
Excessive output truncated after 524395 bytes.

60
k : 14
0.0005
-75805.36309707077
2760.191657835385


In [ ]:
df

In [ ]:
using CSV
CSV.write("Homogeneous ACW-MDP.csv", df)

In [ ]:
Q